Interpolación usando splines cuadráticos

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from sympy.abc import x
import pandas as pd


class GaussJordan:

  def buscarPivote(self, filas, col, M):
      indiceFila = -1
      maxNum = np.inf *-1
      for i in range(col+1, filas):
        if (M[i][col] > maxNum and M[i][col] != 0):
          indiceFila = i
          maxNum = M[i][col]
      return indiceFila

  def intercambiarFilas(self, index1, index2, M):
    for i in range(len(M[0])): 
      tmp = M[index2][i]
      M[index2][i] = M[index1][i]
      M[index1][i] = tmp
    return M

  def gaussJordan(self, matriz):
    n=len(matriz)
    indicePiv=-1
    solucion=1

    for i in range(n):
      pivote=matriz[i][i]
      if (pivote==0):
        indicePiv=self.buscarPivote(n,i,matriz)
        if indicePiv==-1:
          print("El sistema no tiene solución")
          solucion=0
          exit(0)
        else:
          matriz=self.intercambiarFilas(indicePiv,i,matriz)
          pivote=matriz[i][i]
      for j in range(i,n+1):#debe ser n+1 cuando sea matriz aumentada
        matriz[i][j]=matriz[i][j]/pivote
        #imprimirMatriz(matriz)
      for j in range(i+1,n): 
        aux=matriz[j][i]
        for k in range(i,n+1):#debe ser n+1 cuando sea matriz aumentada
          matriz[j][k]=matriz[j][k]-matriz[i][k]*aux
          #imprimirMatriz(matriz)
          
    if solucion==1:
      #print("Gauss")
      #imprimirMatriz(matriz)
      for i in range(n-1,0,-1):
        for j in range(0,i):
          #if (len(M)==len(M[0])):
            #Matriz cuadrada
          #matriz[j][n-1]=matriz[j][n-1]-matriz[i][n-1]*matriz[j][i]
          #else:
            #Matriz aumentada
            matriz[j][n]=matriz[j][n]-matriz[i][n]*matriz[j][i]
        matriz[j][i]=0
      #print("Gauss Jordan")
      #imprimirMatriz(matriz)

class Interpolation:

  #Crea matriz para interpolatión Spline
  def matrizSpline(self,t,v):
    n=len(t)
    nsplines=n-1
    nder=nsplines-1 #numero de derivadas

    splines=[]
    derivatives=[]

    for i in range(nsplines):
      primerSpline=[t[i]**2,t[i],1]
      segundoSpline=[t[i+1]**2,t[i+1],1]
      splines.append(primerSpline)
      splines.append(segundoSpline)

    for i in range(nder):
      eqn=[2*t[i+1],1,0,-2*t[i+1],-1]
      derivatives.append(eqn)

    M=np.zeros((3*n-3,3*n-2))

    paso=-3

    for i in range(len(splines)):
      if (i%2==0):
        paso=paso+3
      for j in range(0,len(splines[0])):
        M[i][j+paso]=splines[i][j]

    paso=-3

    for i in range(len(splines),len(splines)+nder):
      paso=paso+3
      for j in range(len(derivatives[0])): #+1
        M[i][j+paso]=derivatives[i-len(splines)][j]

    M[len(M)-1][0]=1

    paso=0
    for i in range(nsplines):
      M[i+paso][len(M)]=v[i]
      paso=paso+1
      M[i+paso][len(M)]=v[i+1]
    return M

  def saveCoefficients(self,M,numSplines):
    C=[]

    for i in range(len(M)):
      C.append(M[i][-1])

    return C

  def interpolationSpline(self,C):
    functions=[]
    paso=-2

    for i in range(int(len(C)/3)):
      paso=paso+2
      aux=[C[i+paso]*x**2+C[i+1+paso]*x+C[i+2+paso]]
      functions.append(aux)

    return functions

class Graph:

  def plotScatter(self,X,Y):
    plt.scatter(X,Y)
    plt.title("Diagrama de dispersión")
    plt.xlabel("X")
    plt.ylabel("Y")

  def plotLine(self,func,a,b):
    X=np.arange(a,b+0.1,0.1)
    Y=np.zeros_like(X)

    for i in range(len(X)):
      Y[i]=func.subs(x,X[i])

    plt.plot(X,Y)


  def graphSpline(self,functions,X):
    objGraph=Graph()

    for i in range(len(functions)):
      objGraph.plotLine(functions[i][0],X[i],X[i+1])

    
class Test():

  #Regresa dos arreglos con valores aleatorios
  def valuesRandomInt(self,minX,maxX,minRandom,maxRandom):
    #Entradas:minimo valor de X, máximo valor de X
    #         intervalo mínimo y máximo de valores aleatorios
    X=np.arange(minX,maxX+1,1)
    Y=np.zeros_like(X)

    for i in range(len(Y)):
      Y[i]=np.random.randint(minRandom,maxRandom+1)

    return X,Y
    
def main():

  df=pd.read_csv("https://raw.githubusercontent.com/brunolopez941/PracticasComputacionII/main/deaths-and-new-cases-of-hiv.csv")
  df1=df[["Entity","Year","Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Number)"]]
  
  dfNuevo=df1[3668:3696]
  dfNuevo.columns=["País","Año","MuertesPorAño"]
  

  X=dfNuevo.Año.to_numpy()
  Y=dfNuevo.MuertesPorAño.to_numpy()
  
  #objT=Test()
  #X,Y=objT.valuesRandomInt(0,400,-1000,1000)

  start=time.time() #Inicio

  objI=Interpolation()
  M=objI.matrizSpline(X,Y)
  #print("Matriz para Interpolación Spline\n",M)

  objGJ=GaussJordan()
  objGJ.gaussJordan(M)
  
  C=objI.saveCoefficients(M,len(X)-1)
  functions=objI.interpolationSpline(C)

  end=time.time() #Final
  tiempo=end-start #Tiempo total
  print("Tiempo de ejecución:",tiempo)
  print("")

  objG=Graph()
  objG.plotScatter(X,Y)
  objG.graphSpline(functions,X)

  
if __name__=="__main__":
  main()